# 单层网络模型

本小节主要介绍单层神经网络模型的设计，使用单层网络构建训练网络作为讲解实例。


In [5]:
import os
# os.environ['DEVICE_ID'] = '0'
import csv
import numpy as np
from pprint import pprint

import mindspore as ms
from mindspore import nn
from mindspore import context
from mindspore import dataset
from mindspore.train.callback import LossMonitor

context.set_context(mode=context.GRAPH_MODE, device_target="CPU")

创建数据集

In [6]:
def create_dataset(data_path):
    with open(data_path) as csv_file:
        data = list(csv.reader(csv_file, delimiter=','))
    pprint(data[0:5]); pprint(data[50:55]); pprint(data[100:105]) # print some samples

    label_map = {
        'setosa': 0,
        'versicolor': 1,
        'virginica': 2
    }
    X = np.array([[float(x) for x in s[:-1]] for s in data[:150]], np.float32)
    Y = np.array([label_map[s[-1]] for s in data[:150]], np.int32)

    # Split dataset into train set and validation set by 8:2.
    train_idx = np.random.choice(150, 120, replace=False)
    test_idx = np.array(list(set(range(150)) - set(train_idx)))
    X_train, Y_train = X[train_idx], Y[train_idx]
    X_test, Y_test = X[test_idx], Y[test_idx]

    # Convert the data to MindSpore Dataset.
    XY_train = list(zip(X_train, Y_train))
    ds_train = dataset.GeneratorDataset(XY_train, ['x', 'y'])
    ds_train = ds_train.shuffle(buffer_size=120).batch(32, drop_remainder=True)

    XY_test = list(zip(X_test, Y_test))
    ds_test = dataset.GeneratorDataset(XY_test, ['x', 'y'])
    ds_test = ds_test.batch(30)

    return ds_train, ds_test

softmax回归分类模型

In [7]:
def softmax_regression(ds_train, ds_test):
    net = nn.Dense(4, 3)
    loss = nn.loss.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
    opt = nn.optim.Momentum(net.trainable_params(), learning_rate=0.05, momentum=0.9)

    model = ms.train.Model(net, loss, opt, metrics={'acc', 'loss'})
    model.train(25, ds_train, callbacks=[LossMonitor(per_print_times=ds_train.get_dataset_size())], dataset_sink_mode=False)
    metrics = model.eval(ds_test)
    print(metrics)

导入数据集并开始训练

In [8]:
data_url = "iris.csv"

if data_url.startswith('s3'):
    data_path = 'Iris.data'
    import moxing
    moxing.file.copy_parallel(src_url=os.path.join(data_url, 'iris.data'), dst_url=data_path)
else:
    data_path = os.path.abspath(data_url)

softmax_regression(*create_dataset(data_path))

[['5.1', '3.5', '1.4', '0.2', 'setosa'],
 ['4.9', '3', '1.4', '0.2', 'setosa'],
 ['4.7', '3.2', '1.3', '0.2', 'setosa'],
 ['4.6', '3.1', '1.5', '0.2', 'setosa'],
 ['5', '3.6', '1.4', '0.2', 'setosa']]
[['7', '3.2', '4.7', '1.4', 'versicolor'],
 ['6.4', '3.2', '4.5', '1.5', 'versicolor'],
 ['6.9', '3.1', '4.9', '1.5', 'versicolor'],
 ['5.5', '2.3', '4', '1.3', 'versicolor'],
 ['6.5', '2.8', '4.6', '1.5', 'versicolor']]
[['6.3', '3.3', '6', '2.5', 'virginica'],
 ['5.8', '2.7', '5.1', '1.9', 'virginica'],
 ['7.1', '3', '5.9', '2.1', 'virginica'],
 ['6.3', '2.9', '5.6', '1.8', 'virginica'],
 ['6.5', '3', '5.8', '2.2', 'virginica']]
epoch: 1 step: 3, loss is 1.1257935762405396
epoch: 2 step: 3, loss is 0.9012084603309631
epoch: 3 step: 3, loss is 0.6202111840248108
epoch: 4 step: 3, loss is 0.5025111436843872
epoch: 5 step: 3, loss is 0.47775551676750183
epoch: 6 step: 3, loss is 0.36808252334594727
epoch: 7 step: 3, loss is 0.35937026143074036
epoch: 8 step: 3, loss is 0.35549604892730713
